In [1]:
import xarray as xr
import glob
import os
import numpy as np
import xarray as xr
import numpy as np
import os
from dask.diagnostics import ProgressBar

def relclass(landcover_da):

    if not landcover_da.chunks:
        landcover_da = landcover_da.chunk({"time": -1, "lat": 250, "lon": 250})


    reclassed = landcover_da.copy()

#cropland
    cropland_mask = (landcover_da >= 10) & (landcover_da <= 40)
    reclassed = reclassed.where(~cropland_mask, 1)
#forest
    forest_mask = ((landcover_da >= 50) & (landcover_da <= 90)) \
                  | (landcover_da == 160) | (landcover_da == 170)
    reclassed = reclassed.where(~forest_mask, 2)
#savanna
    savanna_mask = (landcover_da == 100) | (landcover_da == 110)
    reclassed = reclassed.where(~savanna_mask, 3)
#shrub
    shrub_mask = (landcover_da == 120) | (landcover_da == 121) | (landcover_da == 122)
    reclassed = reclassed.where(~shrub_mask, 4)
#grassland and arid ecosystem
    grassland_mask = ((landcover_da >= 130) & (landcover_da <= 153))
    reclassed = reclassed.where(~grassland_mask, 5)
#wetland
    wetland_mask = (landcover_da == 180)
    reclassed = reclassed.where(~wetland_mask, 6)
#built-up
    built_up_mask = (landcover_da == 190)
    reclassed = reclassed.where(~built_up_mask, 7)
#bare area and ice
    bare_area_mask = (landcover_da == 200) | (landcover_da == 201) | (landcover_da == 202)
    reclassed = reclassed.where(~bare_area_mask, 8)
#water
    water_mask = (landcover_da == 210)
    reclassed = reclassed.where(~water_mask, 9)

    return reclassed
    

def identify_cropland_abandonment(reclassed_da):
    """
    Open a large netCDF file (with land cover classes), identify cropland abandonment,
    and save the result as a compressed netCDF4 file in C:\\PhDart\\DATA.

    Parameters
    ----------
    input_nc : str
        Path to the input netCDF file containing 'lccs_class'.
    output_dir : str, optional
        Directory where the output netCDF file will be saved (default C:\\PhDart\\DATA).
    output_filename : str, optional
        Output netCDF filename (default 'abandonment_year_computed.nc').
    subset : bool, optional
        Whether to subset the data to a bounding box (lat_bounds, lon_bounds).
    lat_bounds : (float, float), optional
        Min/max latitude for subsetting if subset=True.
    lon_bounds : (float, float), optional
        Min/max longitude for subsetting if subset=True.

    Returns
    -------
    xarray.DataArray
        2D array (lat, lon) of the first year of abandonment or NaN if never abandoned.
        Coordinates, chunking, etc., retained until final compute.
    """
    # 1) Open dataset with netCDF4 engine and chunk
    if not reclassed_da.chunks:
        reclassed_da = reclassed_da.chunk({'time': -1, 'lat': 250, 'lon': 250})

    # 2) Define conditions:
    #    - Cropland = 1, Built-up = 7
    #    - Must be cropland in first year (time=0)
    cropland_initial_mask = (reclassed_da.isel(time=0) == 1)

    # 3) Non-cropland means not 1 and not 7
    is_non_cropland = (reclassed_da != 1) & (reclassed_da != 7)

    # 4) Rolling 5-year window
    consecutive_5 = (
        is_non_cropland
        .rolling(time=5, min_periods=5)
        .construct("window_dim")
        .all("window_dim")
    )

    # 5) Previous year must be cropland
    prev_year_cropland = (reclassed_da.shift(time=1) == 1)

    # 6) Combine conditions; fill NaNs with False
    abandonment_condition = (consecutive_5 & prev_year_cropland).fillna(False)

    # 7) Identify if a pixel ever meets the condition
    ever_abandoned = abandonment_condition.any(dim="time")

    # 8) idxmax gives the first time coordinate where True occurs
    first_abandonment_datetime = abandonment_condition.idxmax(dim="time")

    # Where condition never occurs or was never cropland initially, set to NaT
    mask = ever_abandoned & cropland_initial_mask
    first_abandonment_datetime = first_abandonment_datetime.where(mask, np.datetime64("NaT"))

    # 9) Convert datetime -> year (float). Fill missing with 0, then revert 0 to NaN
    abandonment_year = first_abandonment_datetime.dt.year
    abandonment_year = abandonment_year.astype(float).fillna(0)
    abandonment_year = abandonment_year.where(abandonment_year > 0, np.nan)

    return abandonment_year.rename("abandonment_year")

lc = xr.open_dataset(
    r'data\merged_lccs.nc',
    engine="netcdf4",
    chunks={"time": -1, "lat":100, "lon": 100}
)
reclassed_da = relclass(lc['lccs_class'])
output_path = r'output\merged_lccs.nc'

with ProgressBar():
    reclassed_da.compute()
    
reclassed_da.to_netcdf(
    output_path,
    engine="netcdf4",
    format="NETCDF4",
    encoding={"lccs_class": {"zlib": True, "complevel": 4}}
)



print("Saved the reclassed result!")


'''
abandonment_year_da = identify_cropland_abandonment(reclassed_da)
with ProgressBar():
    abandonment_year_computed = abandonment_year_da.compute()

abandonment_year_computed.to_netcdf(
    "C:/PhDart/DATA/abandonment_year_computed_V1.nc",
    engine="netcdf4",
    format="NETCDF4",
    encoding={"abandonment_year": {"zlib": True, "complevel": 4}}
)
print("Saved the final result!")
'''



c:\Users\zpy10\anaconda3\envs\glbcropland\Lib\site-packages\xarray\core\dataset.py:271: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
c:\Users\zpy10\anaconda3\envs\glbcropland\Lib\site-packages\xarray\core\dataset.py:271: UserWarning: The specified chunks separate the stored chunks along dimension "lon" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


KeyboardInterrupt: 

In [7]:
lc

<xarray.Dataset> Size: 260GB
Dimensions:     (time: 31, lat: 64800, lon: 129600)
Coordinates:
  * lat         (lat) float64 518kB 90.0 90.0 89.99 89.99 ... -89.99 -90.0 -90.0
  * lon         (lon) float64 1MB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * time        (time) datetime64[ns] 248B 1992-01-01 1993-01-01 ... 2022-01-01
Data variables:
    lccs_class  (time, lat, lon) uint8 260GB dask.array<chunksize=(31, 250, 250), meta=np.ndarray>
Attributes: (12/38)
    id:                         ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992-v2.0.7cds
    title:                      Land Cover Map of ESA CCI brokered by CDS
    summary:                    This dataset characterizes the land cover of ...
    type:                       ESACCI-LC-L4-LCCS-Map-300m-P1Y
    project:                    Climate Change Initiative - European Space Ag...
    references:                 http://www.esa-landcover-cci.org/
    ...                         ...
    geospatial_lon_max:         180
    spatial_resolution:         300m
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.002778
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.002778

In [ ]:
rc = xr.open_dataset(
    output_path,
    engine="netcdf4",
    chunks={"time": -1, "lat":250, "lon": 250}
)
abandonment_year_da = identify_cropland_abandonment(rc['lccs_class'])
with ProgressBar():
    abandonment_year_computed = abandonment_year_da.compute()

abandonment_year_computed.to_netcdf(
    "C:/PhDart/DATA/abandonment_year_computed_V1.nc",
    engine="netcdf4",
    format="NETCDF4",
    encoding={"abandonment_year": {"zlib": True, "complevel": 4}}
)
print("Saved the final result!")

# 全球撂荒耕地对气候行动的潜在贡献被显著低估
# 应该慎重考虑耕地撂荒再开发以避免可持续发展目标or人类福祉受损

ceteris paribus(to control the other factors unvaried)

Agricultutal land abandonment: The FAO defines as agricultual land a portion of land dedicated to food production, and if this condition ceases for a period longer than 5 years, the land is considered abandoned.


## 1. cropland abandonment(1992-2022):
firstly, combine 31 datasets into one, this step is to make sure the data is time continuous. 

core aim: to identify the cropland abandonment, the abdonment could be defined as rules as:

caculation 1: expansion cropland but persistent abandonment  
caculation 2: stable cropland and persistent abandonment 
caculation 3: abandonment and recultivation (abandonment and built-up should alse be excluded)

before calculation, the data should be reclassified to 1992-2022, and the reclassification rules should follow the following rules[1](https://www.nature.com/articles/s41597-020-00599-8#:~:text=Independently%20validating%20a%20glob):

1. forest(50:50-90,160,170)
2. savanna(100,110)
3. cropland(10:10-40)
4. built-up(190)
5. wetland(180)
6. shrub(120:120-122)
7. arid ecosystem(150:140,150-153)
8. bare area and ice(200:200-202)
9. water(210)
10. grassland(130)

===========================================================
small calculation:
1) in calculation 1, to see the interval of expansion pixels and abandonment, the first time of expansion, the first time of abandonment; 
2) in calculation 3, to see the persistence of abandonment;
3) in calculation 2, to see the first time of cropland abandonment;

(each condition should be independent)

--->---calcluation 1: expansion cropland but persistent abandonment

aim: to see if the cropland expansion hotspot shift to Global South and to see if the hotspot transformation is due to the Global North cropland abandonment.

1) _**effective change**_:change to cropland and remain for at least 2 years;
2) _**persistence**_: after identified as abandonment, it remains non-cropland (excluding built-up area)
3) _**long-term non-cropland**_: GLC_FCS data using majority method to resample to 1km resolution. If the pixel in 1985 and 1990 are non-cropland, but it changed within the first 5 years (convert for 2 consecutive years like ), it could also be considered as cropland expansion. (to avoid underminishing the cropland expansion). According to [referencen], not consider 1985 and 1990 cropland.

--->---calculation 2: stable cropland and persistent abandonment 

aim: some reseach only consider the initial cropland, however, the immigration and scoical-economic fators may accelerate the cropland abandonment.


1) _**effecitive abandon**_:the consecutive 5 years of non-cropland (exclude built-up area);
2) _**persistence**_: it should remain as abandonment 
3) _**stable cropland**_:GLC_FCS to ensure the effectiveness of abandonment. If a conversion happens within the first 5 years, and 1985 and 1990 are cropland, it should be considered as cropland abandonment;

--->---calculation 3: abandonment and recultivation

aim: this situation is considered as special situation, to see with time-series data, 

1) _**effective abandon**_: it should be identified as cropland abandonment
2) _**reclutivate change**_: the pixel turns back to cropland for two consecutive years
3) _**persitence**_: the first time is just OK



## 2. reclassification and biodiversity:






## Nexus approach of analyzing complex interlinkages


